In [7]:
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import random

device = 'cuda' if torch.cuda.is_available() else 'cpu'

# for reproducibility
random.seed(777)
torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

In [8]:
learning_rate = 0.001
training_epochs = 15
batch_size = 100

In [9]:
mnist_train = dsets.MNIST(root='MNIST_data/',
                          train=True,
                          transform=transforms.ToTensor(),
                          download=True)

mnist_test = dsets.MNIST(root='MNIST_data/',
                         train=False,
                         transform=transforms.ToTensor(),
                         download=True)

In [10]:
data_loader = torch.utils.data.DataLoader(dataset=mnist_train,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          drop_last=True)

In [11]:
linear = torch.nn.Linear(784, 10).to(device)

In [18]:
linear.weight

Parameter containing:
tensor([[-2.9863e-02, -6.3304e-04, -6.9076e-03,  ..., -1.8779e-02,
          3.0876e-02, -3.1737e-02],
        [ 6.7273e-03,  2.0005e-02, -2.4505e-02,  ..., -2.7296e-02,
         -3.8136e-03,  1.1463e-03],
        [-1.7847e-02, -2.9822e-02,  7.4010e-05,  ..., -3.2111e-02,
          2.6378e-02,  3.2280e-02],
        ...,
        [-1.3945e-02,  3.2514e-02,  2.6534e-03,  ..., -3.2780e-02,
         -1.6748e-02, -5.4733e-03],
        [ 1.0081e-02, -1.2429e-02, -1.0554e-02,  ..., -2.0057e-02,
         -9.3248e-04,  8.3561e-03],
        [-6.6728e-03,  7.1139e-03,  2.5636e-02,  ...,  1.9851e-02,
         -1.8774e-02, -3.2610e-02]], requires_grad=True)

In [19]:
linear.weight[0][0]

tensor(-0.0299, grad_fn=<SelectBackward>)

In [6]:
torch.nn.init.normal_(linear.weight)

Parameter containing:
tensor([[-0.1953, -0.4404,  0.0139,  ...,  0.7510,  1.5190,  0.9637],
        [ 0.4146, -1.0660, -0.9969,  ..., -0.3259, -0.2695, -1.3316],
        [ 0.2600, -1.1821,  0.0065,  ...,  1.0648, -1.2251,  0.9841],
        ...,
        [ 0.0279, -1.7242, -1.1723,  ...,  1.3651, -0.0689, -0.7931],
        [ 0.0952,  0.1939,  0.1636,  ..., -1.0489,  0.6224, -1.7181],
        [ 2.6504,  0.6297, -1.6571,  ..., -0.0781, -1.1398, -0.8711]],
       requires_grad=True)

In [26]:
criterion = torch.nn.CrossEntropyLoss().to(device)    # Softmax is internally computed.
optimizer = torch.optim.Adam(linear.parameters(), lr=learning_rate)

In [29]:
total_batch = len(data_loader)
for epoch in range(training_epochs):
    avg_cost = 0

    for X, Y in data_loader:
        # reshape input image into [batch_size by 784]
        # label is not one-hot encoded
        X = X.view(-1, 28 * 28).to(device)
        Y = Y.to(device)

        optimizer.zero_grad()
        hypothesis = linear(X)
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()

        avg_cost += cost / total_batch

    print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.9f}'.format(avg_cost))

print('Learning finished')

Epoch: 0001 cost = 0.606645226
Epoch: 0002 cost = 0.342930645
Epoch: 0003 cost = 0.307940185
Epoch: 0004 cost = 0.291708708
Epoch: 0005 cost = 0.281097472
Epoch: 0006 cost = 0.274351507
Epoch: 0007 cost = 0.269009680
Epoch: 0008 cost = 0.265052646
Epoch: 0009 cost = 0.261682451
Epoch: 0010 cost = 0.258949131
Epoch: 0011 cost = 0.256552845
Epoch: 0012 cost = 0.254249007
Epoch: 0013 cost = 0.252861887
Epoch: 0014 cost = 0.250864118
Epoch: 0015 cost = 0.249818280
Learning finished


In [44]:
with torch.no_grad():
    X_test = mnist_test.test_data.view(-1, 28 * 28).float().to(device)
    Y_test = mnist_test.test_labels.to(device)

    prediction = linear(X_test)
    correct_prediction = torch.argmax(prediction, 1) == Y_test
    accuracy = correct_prediction.float().mean()
    print('Accuracy:', accuracy.item())

    # Get one and predict
    r = random.randint(0, len(mnist_test) - 1)
    X_single_data = mnist_test.test_data[r:r + 1].view(-1, 28 * 28).float().to(device)
    Y_single_data = mnist_test.test_labels[r:r + 1].to(device)

    print('Label: ', Y_single_data.item())
    single_prediction = linear(X_single_data)
    print('Prediction: ', torch.argmax(single_prediction, 1).item())

C:\Users\dxodn\Anaconda3\lib\site-packages\torchvision\datasets\mnist.py:58: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
C:\Users\dxodn\Anaconda3\lib\site-packages\torchvision\datasets\mnist.py:48: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")


Accuracy: 0.899399995803833
Label:  8
Prediction:  8


In [35]:
torch.argmax(prediction, 1)

tensor([7, 2, 1,  ..., 4, 5, 6])

In [34]:
correct_prediction

tensor([True, True, True,  ..., True, True, True])